In [1]:
import asyncio
from ldp.agent import SimpleAgent
import nest_asyncio
from aviary.core import Message, Tool,FunctionInfo

from pydantic import BaseModel, Field

/Users/apple/opt/anaconda3/envs/paperqa_env/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [2]:
nest_asyncio.apply()

In [3]:
# Define input/output types for the calculator
class CalculatorInput(BaseModel):
    x: float = Field(description="First number")
    y: float = Field(description="Second number")

class CalculatorOutput(BaseModel):
    result: float = Field(description="The sum of x and y")


In [4]:
async def test_simple_agent():
    # Define the calculator function
    async def calculator(input_data: CalculatorInput) -> CalculatorOutput:
        return CalculatorOutput(result=input_data.x + input_data.y)

    # Create the tool properly structured
    function_info = FunctionInfo(
        name="add",
        description="Add two numbers together",
        parameters=CalculatorInput.model_json_schema(),
        function=calculator,
        input_type=CalculatorInput,
        output_type=CalculatorOutput
    )

    calculator_tool = Tool(
        name="calculator",
        description="A simple calculator that can add two numbers",
        function=function_info
    )

    # Initialize the agent with the tool
    agent = SimpleAgent(
        llm_model={
            "model": "gpt-4o-mini",
            "temperature": 0.1
        },
        sys_prompt="You are a helpful assistant that can perform calculations."
    )

    # Initialize agent state with tools
    state = await agent.init_state(tools=[calculator_tool])

    # Create a test message
    test_message = Message(
        role="user",
        content="Can you help me add 5 and 3?"
    )

    # Get agent's action and new state
    action, new_state, cost = await agent.get_asv(state, [test_message])
    
    # Print results
    print("Action:", action.value.content)
    print("Cost:", cost)
    print("Total tokens used:", new_state.total_tokens)
    print("Total cost so far:", new_state.total_cost)

    return action, new_state, cost


In [5]:
# For Jupyter environment:
await test_simple_agent()

Action: None
Cost: 1.8e-05
Total tokens used: {'input': 27, 'output': 3}
Total cost so far: 1.8e-05


(OpResult(op=ldp.graph.common_ops.LLMCallOp:_llm_call_op, call_id=run_id=UUID('6fc78cdc-0556-417c-8bbf-1fd65af0cd89') fwd_id=UUID('81de7420-8851-4ad0-b259-1673a3aa9f56'), value=ToolRequestMessage(role='assistant', content=None, function_call=None, tool_calls=[ToolCall(id='call_aysXvKcEWGHaXUgWxLpVHsHR', type='function', function=ToolCallFunction(arguments={'x': 5, 'y': 3}, name='add'))])),
 SimpleAgentState(tools=[Tool(type='function', info=FunctionInfo(name='add', description='Add two numbers together', parameters=Parameters(type='object', properties={'x': {'description': 'First number', 'title': 'X', 'type': 'number'}, 'y': {'description': 'Second number', 'title': 'Y', 'type': 'number'}}, required=['x', 'y'], title='CalculatorInput')))], messages=[Message(role='user', content='Can you help me add 5 and 3?'), ToolRequestMessage(role='assistant', content=None, function_call=None, tool_calls=[ToolCall(id='call_aysXvKcEWGHaXUgWxLpVHsHR', type='function', function=ToolCallFunction(argume